In [3]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown
from dotenv import load_dotenv
load_dotenv()

True

### Dataset is 1,200 DS related job posting from Indeed

In [4]:
df = pd.read_parquet("cluster_test_results.parquet")

In [5]:
print(df.shape)
df.head(2)

(1200, 15)


,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label,llm_title,topic_title
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4,"""Join our dynamic team of analysts and data sc...",Data Science and Strategy Manager
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,...",2,"""Experienced Cloud Migration Specialist for SA...",IT Specialist


In [6]:
from langchain.prompts import PromptTemplate

In [7]:
outline_prompt_string = """Context: {text}
As an experienced legal analyst, review the Context which is part of an insurance policy document related to the question.
{question}

Identify Context excerpts that are of specific material importance to the question. 
For excerpts indentified, explain their specific relevancy to the question.
"""

OUTLINE_PROMPT = PromptTemplate(input_variables=["text", "question"], template=outline_prompt_string)

In [8]:
final_prompt_string = """Context: {text}
As an experienced legal analyst, use the Context and the question:
{question}

Compose a case brief using only the Context. 
Working step by step, organize the Context into a well-structured 
outline with the following sections:

Facts of Loss: a concise sumary of facts and events relevant to this claim.
Coverage Issue: [the facts of the coverage question]
Analysis: subsections
   ["Policy excerpts": excerpts from the most relevant policy subsections, 
   "Applications": explanation of how the policy excerpt impacts the question]
Takeaway: the most important policy section for the question.
TODO: optional list of knowledge gaps, or follow-up questions.
"""
FINAL_PROMPT = PromptTemplate(input_variables=["text", "question"], template=final_prompt_string)

In [9]:
from llama_index import SimpleDirectoryReader
from langchain.text_splitter import CharacterTextSplitter

sample_ho3_policy_docs = SimpleDirectoryReader(input_files=['./data//HO3_sample.pdf']).load_data()

In [10]:
docs = []

for i in range(len(sample_ho3_policy_docs)):
    doc = sample_ho3_policy_docs[i].text
    docs.append(doc)

In [11]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000, chunk_overlap=0, separator="\n\n")

In [20]:
policy_text = "\n\n".join(docs)

In [21]:
policy_chunks = text_splitter.split_text(policy_text)

In [22]:
len(policy_chunks)

12

In [23]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=OUTLINE_PROMPT
)

In [24]:
question = """
    The insured was away on vacation and rented the dwelling to a tenant. When the insured came home 
    he found a ceiling collapsed due to a pipe leak in the attic causing significant damage. 
    The damage was hidden from the insured, but the tenant was aware of it. Is this a covered claim?
"""

In [25]:
all_res = []

for i in range(len(policy_chunks)):
    res = llm_chain.predict(text=policy_chunks[i], question=question)
    all_res.append(res)

### GPT-3.5-turbo: Intermediate Output

In [26]:
Markdown(f"{all_res[3]}")

Excerpt 8a states that collapse is defined as an abrupt falling down or caving in of a building or any part of a building that renders it uninhabitable. In this case, the ceiling collapse would be considered a collapse under this definition.

Excerpt 8b states that the insurance policy covers direct physical loss to covered property involving collapse if it was caused by certain perils, including the perils insured against under Coverage C. If the pipe leak in the attic caused the collapse of the ceiling, it would be considered a covered claim under this provision.

Excerpt 8c states that loss to certain property, such as an awning or foundation, is not included under the coverage for collapse unless it is a direct result of the collapse of a building or part of a building. Since the damage in this case is to the ceiling, which is part of the building, it would be included under the coverage for collapse.

Based on these excerpts, it can be concluded that the damage caused by the ceiling collapse due to the pipe leak would be considered a covered claim under the insurance policy.

In [27]:
outline_text = "\n".join(all_res)

In [28]:
final_chain = LLMChain(
    llm=llm,
    prompt=FINAL_PROMPT
)

In [52]:
final_res = final_chain.predict(text=outline_text, question=question)

## GPT-3.5-turbo: Final Answer

In [ ]:
Markdown(f"{final_res}")

Case Brief:

Facts of Loss:
- The insured was away on vacation and rented the dwelling to a tenant.
- Upon returning home, the insured discovered a collapsed ceiling due to a pipe leak in the attic, causing significant damage.
- The damage was hidden from the insured, but the tenant was aware of it.

Coverage Issue:
- The question is whether this claim is covered under the insurance policy.

Analysis:

Policy Excerpts:
- Definition 5.a: "Insured" means the named insured, their spouse, relatives, or other persons under the age of 21 in their care.
- Definition 6.a: "Insured location" means the residence premises, which is defined in Definition 11.a as the one family dwelling where the insured resides.
- Definition 11.a: "Residence premises" means the one family dwelling where the insured resides.
- Coverage B - Other Structures: Covers structures connected to the dwelling by a fence, utility line, or similar connection.
- Coverage C - Personal Property: Covers personal property owned or used by an insured anywhere in the world, as well as personal property owned by others while on the residence premises.

Applications:
- The insured, their spouse, and any relatives or persons under the age of 21 in their care are considered insured under the policy.
- The residence premises, which is the one family dwelling where the insured resides, is the insured location.
- The collapsed ceiling in the dwelling may be considered an "other structure" connected to the dwelling, potentially covered under Coverage B.
- The personal property owned by the insured, which was damaged while the insured was away on vacation, may be covered under Coverage C.
- Additionally, personal property owned by others while on the residence premises may also be covered under Coverage C.

Takeaway:
- The most important policy section for this question is Coverage C - Personal Property, as it covers personal property owned or used by an insured anywhere in the world, as well as personal property owned by others while on the residence premises.

TODO: None.

In [ ]:
llm = OpenAIChat(model="gpt-4", temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=OUTLINE_PROMPT
)

In [ ]:
all_res = []

for i in range(len(policy_chunks)):
    res = llm_chain.predict(text=policy_chunks[i], question=question)
    all_res.append(res)

### GPT-4: Intermediate Output

In [ ]:
Markdown(f"{all_res[3]}")

Excerpt 1: "We will pay up to $1,000 for your share of loss assessment charged during the policy period against you, as owner or tenant of the "residence premises", by a corporation or association of property owners. The assessment must be made as a result of direct loss to property, owned by all members collectively, of the type that would be covered by this policy if owned by you, caused by a Peril Insured Against under Coverage A, other than: (1)Earthquake; or (2)Land shock waves or tremors before, during or after a volcanic eruption."

Relevancy: This excerpt is relevant because it outlines the conditions under which the insurance company will pay for a loss assessment charged against the policyholder. In this case, the policyholder is the owner of the residence premises and the loss is due to a peril insured against under Coverage A, which presumably includes water damage from a pipe leak.

Excerpt 2: "Collapse means an abrupt falling down or caving in of a building or any part of a building with the result that the building or part of the building cannot be occupied for its current intended purpose."

Relevancy: This excerpt is relevant because it defines what constitutes a "collapse" under the policy. The situation described in the question involves a ceiling collapse, which would fall under this definition as it has resulted in the building being unable to be occupied for its current intended purpose.

Excerpt 3: "We insure for direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by one or more of the following: (2)Decay that is hidden from view, unless the presence of such decay is known to an "insured" prior to collapse;"

Relevancy: This excerpt is relevant because it specifies that the insurance company will cover losses involving the collapse of a building or part of a building caused by decay that is hidden from view, unless the insured was aware of the decay prior to the collapse. In the situation described in the question, the insured was not aware of the pipe leak (which could be considered a form of decay) prior to the collapse, but the tenant was. This could potentially complicate the claim, as it would need to be determined whether the tenant's knowledge of the issue should be considered equivalent to the insured's knowledge.

In [ ]:
final_chain = LLMChain(
    llm=llm,
    prompt=FINAL_PROMPT
)

outline_text = "\n".join(all_res)

In [ ]:
final_res = final_chain.predict(text=outline_text, question=question)

## GPT-4:

In [ ]:
Markdown(f"{final_res}")

Facts of Loss: 
The insured was away on vacation and rented the dwelling to a tenant. Upon returning home, the insured discovered that a ceiling had collapsed due to a pipe leak in the attic, causing significant damage. The tenant was aware of the damage, but it was hidden from the insured.

Coverage Issue: 
The question is whether the damage caused by the pipe leak in the attic, which led to the collapse of the ceiling, is covered under the insured's policy.

Analysis: 

Policy Excerpts: 
1. Definition 5.a: "Insured" means the named insured and their spouse, as well as any relatives or other persons under the age of 21 in their care.
2. Definition 6.a: "Insured location" means the residence premises, which is defined in Definition 11.a as the one family dwelling where the insured resides.
3. Coverage B – Other Structures: Structures connected to the dwelling by only a fence, utility line, or similar connection are covered.
4. Coverage C – Personal Property: Personal property owned or used by an "insured" while it is anywhere in the world is covered.
5. Peril Insured Against 13: Sudden and Accidental Tearing Apart, Cracking, Burning or Bulging.
6. Q. Concealment Or Fraud: No coverage if an "insured" has intentionally concealed or misrepresented any material fact or circumstance.
7. Exclusion E.2.b. (2): The rental or holding for rental of an "insured location" is covered, as long as it is used only as a residence and not for any "business" purpose.
8. Exclusion E.4: No coverage for "bodily injury" or "property damage" arising out of a premises owned by an "insured" that is not an "insured location".

Applications: 
The policy provides coverage for the insured and their property, including structures connected to the dwelling and personal property owned or used by the insured. The damage caused by the pipe leak in the attic, which led to the collapse of the ceiling, falls under the peril insured against sudden and accidental tearing apart, cracking, burning or bulging. However, the policy also states that there is no coverage if the insured has intentionally concealed or misrepresented any material fact or circumstance. In this case, the tenant was aware of the damage, but the insured was not. This could be seen as a form of intentional concealment or misrepresentation, which would make the claim ineligible for coverage. The policy also excludes coverage for property damage arising out of a premises owned by an "insured" that is not an "insured location". However, the rental of the dwelling to the tenant is covered under the policy, as long as it is used only as a residence and not for any "business" purpose.

Takeaway: 
The most important policy section for this question is Q. Concealment Or Fraud, which states that there is no coverage if an "insured" has intentionally concealed or misrepresented any material fact or circumstance. This could potentially make the claim ineligible for coverage.

TODO: 
Further investigation is needed to determine whether the tenant's knowledge of the damage and failure to inform the insured constitutes intentional concealment or misrepresentation. Additionally, it would be helpful to know whether the dwelling was used only as a residence and not for any "business" purpose during the rental period.

In [39]:
question = """
    The insured rented the home through AirB&B for several months. When the tenants moved out
    he found they had converted the dwelling into a marijuana grow operation. This resulted in water damage and an
    electrical fire. Is there coverage for the dwelling dweling damage, and do any named perils apply to the personal property damage?
"""

In [40]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=OUTLINE_PROMPT
)

all_res = []
for i in range(len(policy_chunks)):
    res = llm_chain.predict(text=policy_chunks[i], question=question)
    all_res.append(res)

In [41]:
outline_text = "\n".join(all_res)

In [43]:
final_chain = LLMChain(
    llm=llm,
    prompt=FINAL_PROMPT
)
final_res = final_chain.predict(text=outline_text[:15000], question=question)

In [44]:
Markdown(f"{all_res[3]}")

Excerpt 8a: "Collapse means an abrupt falling down or caving in of a building or any part of a building with the result that the building or part of the building cannot be occupied for its current intended purpose."
Relevancy: This excerpt defines what constitutes a collapse, which is important in determining whether the damage caused by the water and electrical fire would be covered under the policy.

Excerpt 8b: "We insure for direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by one or more of the following: (1) The Perils Insured Against named under Coverage C; (2) Decay that is hidden from view, unless the presence of such decay is known to an 'insured' prior to collapse; (3) Insect or vermin damage that is hidden from view, unless the presence of such damage is known to an 'insured' prior to collapse; (4) Weight of contents, equipment, animals or people; (5) Weight of rain which collects on a roof; or (6) Use of defective material or methods in construction, remodeling or renovation if the collapse occurs during the course of the construction, remodeling or renovation."
Relevancy: This excerpt explains the specific circumstances under which coverage for collapse is provided. It states that coverage is provided if the collapse was caused by one of the named perils under Coverage C, which may include water damage and electrical fire.

Excerpt 11a: "You may use up to 10% of the limit of liability that applies to Coverage A for the increased costs you incur due to the enforcement of any ordinance or law which requires or regulates: (1) The construction, demolition, remodeling, renovation or repair of that part of a covered building or other structure damaged by a Peril Insured Against; (2) The demolition and reconstruction of the undamaged part of a covered building or other structure, when that building or other structure must be totally demolished because of damage by a Peril Insured Against to another part of that covered building or other structure; or (3) The remodeling, removal or replacement of the portion of the undamaged part of a covered building or other structure necessary to complete the remodeling, repair or replacement of that part of the covered building or other structure damaged by a Peril Insured Against."
Relevancy: This excerpt explains that coverage may be provided for increased costs incurred due to the enforcement of any ordinance or law related to the construction, demolition, remodeling, renovation, or repair of a covered building or structure damaged by a covered peril. It may be relevant in determining whether there is coverage for the repairs needed after the water damage and electrical fire.

In [45]:
Markdown(f"{final_res}")

Facts of Loss:
- The insured rented the home through AirB&B for several months.
- The tenants converted the dwelling into a marijuana grow operation.
- This resulted in water damage and an electrical fire.

Coverage Issue:
- Is there coverage for the dwelling damage caused by the water damage and electrical fire?
- Do any named perils apply to the personal property damage?

Analysis:
1. Policy excerpts:
   - Excerpt 1: "We will provide the insurance described in this policy in return for the premium and compliance with all applicable provisions of this policy."
      - Application: This excerpt establishes that coverage will be provided as long as the insured complies with the policy provisions.
   - Excerpt 2: "Liability for 'property damage' arising out of the...occupancy, operation, use, loading or unloading of such vehicle or craft by any person."
      - Application: This excerpt suggests that there may be coverage for the property damage caused by the tenants' conversion of the dwelling into a marijuana grow operation, as it falls under the category of property damage arising out of the occupancy or use of the property.
   - Excerpt 3: "Occurrence' means an accident, including continuous or repeated exposure to substantially the same general harmful conditions, which results, during the policy period, in... 'property damage'."
      - Application: This excerpt defines the term "occurrence" and includes property damage as one of the results of an occurrence. It indicates that if the water damage and electrical fire were accidental and occurred during the policy period, they may be considered an occurrence and potentially covered under the policy.
   - Excerpt 4: "Property damage' means physical injury to, destruction of, or loss of use of tangible property."
      - Application: This excerpt provides the definition of property damage, which includes physical injury to tangible property. It suggests that the water damage and electrical fire, which resulted from the tenants' actions, may be considered property damage and potentially covered under the policy.

Takeaway:
- The most important policy section for the question is the definition of property damage, as it establishes that the water damage and electrical fire caused by the tenants' actions may be considered property damage and potentially covered under the policy.

TODO: None.

In [46]:
llm = ChatOpenAI(model="gpt-4", temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=OUTLINE_PROMPT
)

all_res = []
for i in range(len(policy_chunks)):
    res = llm_chain.predict(text=policy_chunks[i], question=question)
    all_res.append(res)


In [47]:
outline_text = "\n".join(all_res)

In [48]:
final_chain = LLMChain(
    llm=llm,
    prompt=FINAL_PROMPT
)
final_res = final_chain.predict(text=outline_text, question=question)

In [50]:
Markdown(f"{all_res[1]}")

Excerpt 1: "We cover: a.The dwelling on the "residence premises" shown in the Declarations, including structures attached to the dwelling; and b.Materials and supplies located on or next to the "residence premises" used to construct, alter or repair the dwelling or other structures on the "residence premises"."
Relevancy: This excerpt is relevant because it outlines the coverage for the dwelling, which is the subject of the question. The damage caused by the tenants converting the dwelling into a marijuana grow operation would fall under this coverage.

Excerpt 2: "We do not cover: a.Land, including land on which the other structures are located; b.Other structures rented or held for rental to any person not a tenant of the dwelling, unless used solely as a private garage; c.Other structures from which any "business" is conducted; or d.Other structures used to store "business" property."
Relevancy: This excerpt is relevant because it outlines what is not covered under the policy. The fact that the dwelling was rented out through AirB&B could potentially be considered as a "business" activity, which might exclude the damage from coverage.

Excerpt 3: "We cover personal property owned or used by an "insured" while it is anywhere in the world."
Relevancy: This excerpt is relevant because it outlines the coverage for personal property. The question asks about coverage for personal property damage, which would fall under this provision.

Excerpt 4: "We do not cover: a.Articles separately described and specifically insured, regardless of the limit for which they are insured, in this or other insurance; b.Animals, birds or fish; c."Motor vehicles"."
Relevancy: This excerpt is relevant because it outlines what personal property is not covered under the policy. Depending on what personal property was damaged in the electrical fire, this provision could potentially exclude certain items from coverage. 

Excerpt 5: "h.Property rented or held for rental to others off the "residence premises"; i."Business" data, including such data stored in: (1)Books of account, drawings or other paper records; or (2)Computers and related equipment."
Relevancy: This excerpt is relevant because it outlines exclusions for property rented or held for rental and for business data. If the dwelling was considered "rented or held for rental" or if any business data was damaged, these exclusions could apply.

In [51]:
Markdown(f"{final_res}")

Facts of Loss:
The insured rented out their home through AirB&B for several months. After the tenants moved out, the insured discovered that they had converted the dwelling into a marijuana grow operation. This resulted in water damage and an electrical fire. The insured is seeking coverage for the dwelling damage and personal property damage.

Coverage Issue:
The main issue is whether the insurance policy provides coverage for the dwelling damage and personal property damage caused by the tenants' marijuana grow operation.

Analysis:
Policy Excerpts:
1. "We will provide the insurance described in this policy in return for the premium and compliance with all applicable provisions of this policy."
2. "Any part of a premises occasionally rented to an "insured" for other than "business" use."
3. "A trade, profession or occupation engaged in on a full-time, part-time or occasional basis; or Any other activity engaged in for money or other compensation."
4. "Property damage" means physical injury to, destruction of, or loss of use of tangible property."
5. "Occurrence" means an accident, including continuous or repeated exposure to substantially the same general harmful conditions, which results, during the policy period, in: a."Bodily injury"; or b."Property damage".

Applications:
The policy provides coverage for property damage resulting from an "occurrence". The water damage and electrical fire caused by the tenants' marijuana grow operation could potentially be considered an "occurrence" under this definition. However, the policy also states that coverage is contingent on the insured's compliance with all policy provisions. If the insured violated any policy terms, such as by renting out the property without notifying the insurer or obtaining the necessary endorsements, coverage may not apply. Furthermore, the policy suggests that it may cover premises that are occasionally rented out for non-business use. However, it's unclear whether renting out the home through AirB&B for several months would be considered "occasional" use or whether it would be considered a business use, which could affect coverage.

Takeaway:
The most important policy section for this question is the definition of "occurrence" and the provision regarding compliance with all policy terms. Whether the damage is covered will largely depend on whether the tenants' marijuana grow operation is considered an "occurrence" and whether the insured complied with all policy terms.

TODO:
1. Review the entire policy, including any exclusions or endorsements, to determine if there is coverage for damages resulting from illegal activities such as a marijuana grow operation.
2. Determine whether the insured notified the insurer about renting out the property through AirB&B and whether they obtained the necessary endorsements.
3. Clarify the policy's definition of "occasional" use and "business" use to determine whether the rental through AirB&B would affect coverage.